# Calculating Land use change

In [2]:
import os
import ee
import geemap 
import pandas as pd

#ee.Authenticate()
ee.Initialize()

In [3]:
# Define the bounding box of Colombia (using a shapefile uploaded to the assets folder)
colombiaMpios = ee.FeatureCollection('projects/ee-juamiji/assets/Muni')

# Load image collections
dynamicWorld = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
landsat= ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

# Function to clip each image
def clip_to_colombia(image):
    return image.clip(colombiaMpios)

# Apply the clip function to each image in the collections
dynamicWorld_COL = dynamicWorld.map(clip_to_colombia)
landsat_COL = landsat.map(clip_to_colombia)

# Import the Forest loss image
flossHansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

# Import the Primary Tropical Forest cover image collection and filter it for Colombia
primaryForest_ideam = ee.Image("projects/ee-juamiji/assets/SouthAmerica_2001_primary").selfMask()
primaryForest_ideam_COL = primaryForest_ideam.clip(colombiaMpios)

# Clip the Forest loss image to Colombia's boundaries
flossHansen_COL = flossHansen.clip(colombiaMpios)

In [4]:
# Create a mask where fcover is equal to 1.
primaryMask = primaryForest_ideam_COL.eq(1)

# Masking the floss imagery using the primary mask (cover equals 1)
flossHansen_COLmasked = flossHansen_COL.updateMask(primaryMask)

# Select the 'lossyear' band from the masked forest loss image
loss_year = flossHansen_COLmasked.select(['lossyear'])

In [ ]:
year=2015

#Selecting year of interest
dynamicWorld_year = dynamicWorld_COL.filterDate(f'{year}-01-01', f'{year}-12-31').max()

loss_year_mask = loss_year.eq(year)

#Masking the dynamicWorld image using the loss year mask
dynamicWorld_year_infloss = dynamicWorld_year.updateMask(loss_year_mask)

In [10]:
#Defining function to calculate Land change in forest loss areas
def calculate_land_change(feature, year):

    #Selecting year of interest
    dynamicWorld_year = dynamicWorld_COL.filterDate(f'{year}-01-01', f'{year}-12-31').max()

    loss_year_mask = loss_year.eq(year)

    #Masking the dynamicWorld image using the loss year mask
    dynamicWorld_year_infloss = dynamicWorld_year.updateMask(loss_year_mask)

    # Select the 'label' band from the Dynamic World image
    label_band = dynamicWorld_year_infloss.select('label')

    # Define land cover class codes and their names
    land_cover_classes = {
        2: 'grass',
        4: 'crop',
        5: 'shrub',
        6: 'built',
        7: 'bare'
    }

    # Initialize an empty dictionary to hold area images
    area_images = {}

    # Generate area images for each land cover class
    for code, name in land_cover_classes.items():
        mask = label_band.eq(code)
        area_image = mask.multiply(ee.Image.pixelArea()).rename(f'{name}_area')
        area_images[name] = area_image

    # Compute areas by municipality for selected classes
    results = {}
    for name, image in area_images.items():
        area_sum = image.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=feature.geometry(),
            scale=30,
            maxPixels=1e9
        ).get(f'{name}_area')
        results[f'{name}_area_muni'] = area_sum

    # Return a new feature with only the 'lossArea' and 'codmpio' properties
    return ee.Feature(None, {
        'codmpio': feature.get('codmpio'),
        f'crop_area{year}': results['crop_area_muni'],
        f'grass_area{year}': results['grass_area_muni'],
        f'shrub_area{year}': results['shrub_area_muni'],
        f'built_area{year}': results['built_area_muni'],
        f'bare_area{year}': results['bare_area_muni']
    })

In [14]:
year=2017
current_year_results = colombiaMpios.map(lambda feature: calculate_land_change(feature, year))

# Get first 10 features as a list
first_10 = current_year_results.toList(10)

# Loop over and print each one
for i in range(10):
    feature = ee.Feature(first_10.get(i))
    print(f"Feature {i + 1}:\n", feature.getInfo(), "\n")

Feature 1:
 {'type': 'Feature', 'geometry': None, 'id': '00000000000000000263', 'properties': {'bare_area2017': 0, 'built_area2017': 0, 'codmpio': 5002, 'crop_area2017': 0, 'grass_area2017': 0, 'shrub_area2017': 0}} 

Feature 2:
 {'type': 'Feature', 'geometry': None, 'id': '000000000000000002ff', 'properties': {'bare_area2017': 0, 'built_area2017': 0, 'codmpio': 5004, 'crop_area2017': 0, 'grass_area2017': 0, 'shrub_area2017': 0}} 

Feature 3:
 {'type': 'Feature', 'geometry': None, 'id': '00000000000000000006', 'properties': {'bare_area2017': 0, 'built_area2017': 0, 'codmpio': 50006, 'crop_area2017': 0, 'grass_area2017': 0, 'shrub_area2017': 0}} 

Feature 4:
 {'type': 'Feature', 'geometry': None, 'id': '0000000000000000039c', 'properties': {'bare_area2017': 0, 'built_area2017': 0, 'codmpio': 27006, 'crop_area2017': 0, 'grass_area2017': 0, 'shrub_area2017': 0}} 

Feature 5:
 {'type': 'Feature', 'geometry': None, 'id': '00000000000000000187', 'properties': {'bare_area2017': 0, 'built_area